In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

from __future__ import division
from __future__ import print_function
from operator import itemgetter
from itertools import combinations
import time
import os

import tensorflow as tf
import numpy as np
import networkx as nx
import scipy.sparse as sp
from sklearn import metrics

from decagon.deep.optimizer import DecagonOptimizer
from decagon.deep.model import DecagonModel
from decagon.deep.minibatch import EdgeMinibatchIterator
from decagon.utility import rank_metrics, preprocessing

/Users/ravanv/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Train on CPU (hide GPU) due to memory constraints
os.environ['CUDA_VISIBLE_DEVICES'] = ""

# Train on GPU
#os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
#os.environ["CUDA_VISIBLE_DEVICES"] = '0'
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True

np.random.seed(0)

###########################################################
#
# Functions
#
###########################################################


def get_accuracy_scores(edges_pos, edges_neg, edge_type):
    feed_dict.update({placeholders['dropout']: 0})
    feed_dict.update({placeholders['batch_edge_type_idx']: minibatch.edge_type2idx[edge_type]})
    feed_dict.update({placeholders['batch_row_edge_type']: edge_type[0]})
    feed_dict.update({placeholders['batch_col_edge_type']: edge_type[1]})
    rec = sess.run(opt.predictions, feed_dict=feed_dict)

    def sigmoid(x):
        return 1. / (1 + np.exp(-x))

    # Predict on test set of edges
    preds = []
    actual = []
    predicted = []
    edge_ind = 0
    for u, v in edges_pos[edge_type[:2]][edge_type[2]]:
        score = sigmoid(rec[u, v])
        preds.append(score)
        assert adj_mats_orig[edge_type[:2]][edge_type[2]][u,v] == 1, 'Problem 1'

        actual.append(edge_ind)
        predicted.append((score, edge_ind))
        edge_ind += 1

    preds_neg = []
    for u, v in edges_neg[edge_type[:2]][edge_type[2]]:
        score = sigmoid(rec[u, v])
        preds_neg.append(score)
        assert adj_mats_orig[edge_type[:2]][edge_type[2]][u,v] == 0, 'Problem 0'

        predicted.append((score, edge_ind))
        edge_ind += 1

    preds_all = np.hstack([preds, preds_neg])
    preds_all = np.nan_to_num(preds_all)
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    predicted = list(zip(*sorted(predicted, reverse=True, key=itemgetter(0))))[1]

    roc_sc = metrics.roc_auc_score(labels_all, preds_all)
    aupr_sc = metrics.average_precision_score(labels_all, preds_all)
    apk_sc = rank_metrics.apk(actual, predicted, k=50)

    return roc_sc, aupr_sc, apk_sc


def construct_placeholders(edge_types):
    placeholders = {
        'batch': tf.placeholder(tf.int32, name='batch'),
        'batch_edge_type_idx': tf.placeholder(tf.int32, shape=(), name='batch_edge_type_idx'),
        'batch_row_edge_type': tf.placeholder(tf.int32, shape=(), name='batch_row_edge_type'),
        'batch_col_edge_type': tf.placeholder(tf.int32, shape=(), name='batch_col_edge_type'),
        'degrees': tf.placeholder(tf.int32),
        'dropout': tf.placeholder_with_default(0., shape=()),
    }
    placeholders.update({
        'adj_mats_%d,%d,%d' % (i, j, k): tf.sparse_placeholder(tf.float32)
        for i, j in edge_types for k in range(edge_types[i,j])})
    placeholders.update({
        'feat_%d' % i: tf.sparse_placeholder(tf.float32)
        for i, _ in edge_types})
    return placeholders

In [3]:
###########################################################
#
# Load and preprocess data (This is a dummy toy example!)
#
###########################################################

####
# The following code uses artificially generated and very small networks.
# Expect less than excellent performance as these random networks do not have any interesting structure.
# The purpose of main.py is to show how to use the code!
#
# All preprocessed datasets used in the drug combination study are at: http://snap.stanford.edu/decagon:
# (1) Download datasets from http://snap.stanford.edu/decagon to your local machine.
# (2) Replace dummy toy datasets used here with the actual datasets you just downloaded.
# (3) Train & test the model.
####


In [4]:
#bio-decagon-ppi:Protein-protein interaction network
#bio-decagon-targets: Drug-target protein associations
#bio-decagon-targets-all: Drug-target protein associations culled from several curated databases
#bio-decagon-combo:Polypharmacy side effects in the form of (drug A, side effect type, drug B) triples
#bio-decagon-effectcategories: Side effect categories
#bio-decagon-mono:Side effects of individual drugs in the form of (drug A, side effect type) tuples

<h1>Information about the datasets from the paper:</h1>
<h4>The protein-prtoein network:</h4>
number of proteins = 19,085, number of physical interactions = 719,402
<h4>The drug-prtoein network:</h4>
number of proteins = 8,934, number of drugs = 519,022, number of interactions = 8,083,600
<h4>The drug-drug network(individuale):</h4>
number of  drugs = 1,556, number of side effects = 5,868, number of drug-side effect association = 286,399
<h4>The drug-drugn network(combination):</h4>
number of  drug combinations = 63,473, number of side effect types = 1,318, number of drug combination-side effect association = 4,651,131
<h2>Final Network:</h2>
<h3>Number of protiens = 19,085 (paper) ....... Number of protiens = 19,081(ppi data) </h3>
<h3>Number of drugs = 645 (paper).......... Number of drugs = 645 (polypharmacy side effect data (combo))</h3>
<h3>Number of protien-protien edges= 715,612(paper) ....... Number of protien-protien edges= 715,612 (ppi data)</h3>
<h3>Number of drug-drug edges= 4,651,131 (paper) ......... Number of drug-drug edges= 4,649,441 (polypharmacy side effect data (combo)) </h3>
<h3>Number of drug-protien edges= 18,596 (paper) ........ Number of drug-protien edges= 18,690 (Drug-target protein (targets))</h3>

<h2>Reading real datasets</h2>

In [5]:
protein_protein_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-ppi.csv", sep=',',header = 0)
drug_target_protein_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-targets.csv", sep=',',header = 0)
#drug_target_protein_all_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-targets-all.csv", sep=',',header = 0)
polypharmacy_side_effect_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-combo.csv", sep=',',header = 0)
#side_effect_categories_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-effectcategories.csv", sep=',',header = 0)
#side_effect_individuale_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-mono.csv", sep=',',header = 0)


In [6]:
print("size of the protein-protien network:", protein_protein_data.shape)
print("size of the Drug-target protein associations:", drug_target_protein_data.shape)
#print("size of the Drug-target protein associations culled from several curated databases :",drug_target_protein_all_data.shape)
print("size of the Polypharmacy side effects:", polypharmacy_side_effect_data.shape)
#print("size of the Side effect categories:",side_effect_categories_data.shape)
#print("size of the Side effects of individual drugs:",side_effect_individuale_data.shape)

size of the protein-protien network: (715612, 2)
size of the Drug-target protein associations: (18690, 2)
size of the Polypharmacy side effects: (4649441, 4)


In [7]:
val_test_size = 0.1 #changed from 0.05 to 0.1 
#n_genes = 19081 #from protein-protein network 
#n_drugs = 645 #from drug-drug combo network

<h2> Creating a symmetric adjacency matrix for genes from the protein-protein network </h2>  

In [8]:
#I first created a symmetric adjacency dataframe from the protein-protein data to get the number of genes and their degrees.
df_gene1_gene2 = pd.crosstab(protein_protein_data['Gene 1'], protein_protein_data['Gene 2'])
gene_idx = df_gene1_gene2.columns.union(df_gene1_gene2.index)
df_gene1_gene2 = df_gene1_gene2.reindex(index = gene_idx, columns = gene_idx,fill_value=0)#upper triangle of the adjacency matrix
df_gene2_gene1 = pd.crosstab(protein_protein_data['Gene 2'], protein_protein_data['Gene 1'])
df_gene2_gene1 = df_gene2_gene1.reindex(index = gene_idx, columns = gene_idx, fill_value=0)#lower triangle of the adjacency matrix
gene_adj = df_gene2_gene1.add(df_gene1_gene2, fill_value=0)#creates a symmetric adjacency matrix of Gene 1 and Gene 2 by adding upper triangle and lower triangle
gene_degrees = np.array(gene_adj.sum(axis=0)).squeeze()#get the degrees of genes
print("size of the gene-gene adjacecny matrix:",gene_adj.shape, ", number of genes:",len(gene_degrees))

size of the gene-gene adjacecny matrix: (19081, 19081) , number of genes: 19081


In [9]:
n_genes = len(gene_degrees)

In [9]:
indices_genes = list(range(0,len(gene_idx)))
dict_genes = dict(zip(gene_idx, indices_genes))#making a dictionary to map each gene to the index of gene

<h3>Creating an adjacency matrix for the genes from the adjacency dataframe</h3>

In [10]:
gene_adj_mat = np.zeros((n_genes,n_genes)) # matrix of size n_genes * n_genes
for i in range(0,protein_protein_data.shape[0]):#read from the protein-protein file
    gene1 = protein_protein_data.loc[i][0]#read gene1
    gene2 = protein_protein_data.loc[i][1]#read gene2
    #print(gene1,gene2)
    gene1_index = dict_genes.get(gene1)#get the index of gene1 in dictionary
    gene2_index = dict_genes.get(gene2)#get the index of gene2 in dictionary
    #print(gene1_index,gene2_index)
    gene_adj_mat[gene1_index][gene2_index] = 1.0 
    gene_adj_mat[gene2_index][gene1_index] = 1.0

In [11]:
G=nx.from_numpy_matrix(gene_adj_mat)
edges = G.edges(data=True)

In [12]:
def show_graph_with_labels(adjacency_matrix):#function to draw the adjacency graph, I have removed labels here.
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    nx.draw(gr, node_size=500)#labels can be added here
    plt.show()

In [13]:
#show_graph_with_labels(gene_adj_mat)

In [14]:
gene_adj_mat_t = gene_adj_mat.transpose()#transpose of the gene-gene adjacency matrix

<h2> Creating a list of symmetric adjacency matrices for each side effect from the drug-drug network (combo)</h2>

In [ ]:
list_side_effects = []
prev_side_effect = []
for i in range(0,polypharmacy_side_effect_data.shape[0]):#get the list of side effects
    if polypharmacy_side_effect_data.loc[i][2] not in prev_side_effect:
        list_side_effects.append(polypharmacy_side_effect_data.loc[i][2])
        prev_side_effect.append(polypharmacy_side_effect_data.loc[i][2])
print("number of side effects:", len(list_side_effects))          

In [16]:
df_drug1_drug2 = pd.crosstab(polypharmacy_side_effect_data['STITCH 1'], polypharmacy_side_effect_data['STITCH 2'])
drug_idx = df_drug1_drug2.columns.union(df_drug1_drug2.index)#get names of all drugs

In [ ]:
indices_drugs = list(range(0,len(drug_idx)))
dict_drugs = dict(zip(drug_idx, indices_drugs))#create a dictionary to map each drug to its index.
n_drugs = len(drug_idx)
print("number of drugs:",len(drug_idx))

In [18]:
drug_drug_adj_list = []
for i in range(len(list_side_effects)):#for each side effect, create an adjacency matrix
    drug_drug_mat = np.zeros((n_drugs,n_drugs))
    for row_index in range(polypharmacy_side_effect_data.shape[0]):
        if polypharmacy_side_effect_data.loc[row_index][2] == list_side_effects[i]:
            drug_1 = polypharmacy_side_effect_data.loc[row_index][0]
            drug_2 = polypharmacy_side_effect_data.loc[row_index][1] 
            drug_1_index = dict_drugs.get(drug_1)
            drug_2_index = dict_drugs.get(drug_2)
            print(drug_1_index,drug_2_index)
            drug_drug_mat[drug_1_index,drug_2_index] =   drug_drug_mat[drug_2_index,drug_1_index] = 1.
    drug_drug_adj_list.append(sp.csr_matrix(drug_drug_mat))# add the adjacency matrix for the side effect to the list of adj. matrices
drug_degrees_list = [np.array(drug_adj.sum(axis=0)).squeeze() for drug_adj in drug_drug_adj_list]

55 186
456 553
8 295
380 556
20 155
271 449
40 63
334 351
20 391
544 628
157 540
41 345
92 209
8 127
334 420
369 456
337 346
10 374
337 558
20 162
146 346
374 424
345 401
420 456
373 495
40 369
374 556
27 334
80 420
273 567
424 428
126 428
8 556
334 612
164 456
567 612
126 185
20 62
126 380
334 553
139 351
139 334
183 274
337 495
150 322
80 391
271 369
127 380
380 475
424 475
456 543
157 210
149 155
346 567
8 185
273 351
155 157
16 32
345 627
420 553
80 149
295 374
273 334
391 474
32 236
209 632
295 424
27 139
172 322
155 218
27 456
346 456
17 236
126 337
41 209
346 369
337 373
157 218
40 549
177 209
40 458
61 456
80 369
146 430
35 183
183 209
108 157
8 475
271 351
157 635
139 449
233 309
17 32
126 295
271 346
369 391
420 449
177 183
40 553
337 612
271 334
126 556
430 558
449 567
185 297
449 553
369 612
40 273
40 271
40 346
27 553
295 373
20 474
428 556
80 155
155 275
181 551
108 540
139 456
80 612
185 374
185 475
16 430
295 348
271 420
369 449
369 567
185 424
149 474
271 612
67 374
27

222 309
155 221
265 307
61 559
20 420
25 41
20 421
155 199
107 246
181 614
157 410
190 539
66 146
146 510
209 438
137 512
171 270
168 369
109 352
41 322
383 401
334 614
162 627
80 420
424 568
186 624
79 638
17 168
155 619
68 309
209 451
217 561
200 233
37 157
97 210
97 211
335 420
203 295
42 100
162 270
289 336
387 556
270 502
186 521
144 495
88 290
257 274
79 521
167 380
351 425
209 418
162 444
264 369
138 142
138 612
349 374
35 391
270 430
97 393
20 27
108 544
235 556
10 419
127 502
137 426
262 278
10 558
407 433
229 395
97 137
441 510
230 636
186 460
132 556
281 328
410 622
158 262
92 172
352 544
22 25
532 599
209 612
108 632
459 478
32 262
166 190
163 309
134 228
112 297
257 382
40 45
173 206
7 358
173 208
408 525
374 576
92 138
92 139
286 510
367 532
264 374
225 240
225 463
134 378
389 561
40 173
81 226
351 539
344 487
216 395
457 556
186 367
19 510
239 392
67 312
305 374
273 567
271 401
187 307
109 270
126 427
199 467
15 262
371 598
198 510
334 349
336 416
380 448
344 610
259 274

412 512
17 83
17 80
198 474
102 146
273 539
137 205
322 428
61 560
67 410
150 510
217 635
10 322
244 297
246 441
51 127
352 627
407 459
263 290
166 430
127 635
35 62
17 415
66 449
312 374
104 401
510 565
51 559
137 298
137 297
228 599
51 544
142 346
62 162
138 407
52 467
86 199
134 472
184 346
20 109
20 108
309 412
217 289
15 378
444 467
467 533
142 190
210 430
157 186
368 441
287 585
138 346
318 627
155 520
458 524
367 510
336 400
415 541
271 295
17 561
263 382
106 177
104 407
127 412
502 521
22 218
203 209
41 455
50 131
41 95
150 281
22 289
6 146
181 629
185 544
63 452
109 127
351 558
152 467
41 496
41 297
41 298
127 544
126 205
27 558
263 583
155 638
70 447
210 533
263 627
345 393
127 183
127 182
79 568
108 352
186 198
228 232
15 271
15 272
15 273
310 374
346 452
371 576
352 379
214 352
50 88
203 273
139 270
37 158
33 556
298 491
70 199
150 310
185 418
83 106
351 389
235 308
95 313
260 378
378 638
401 510
272 449
42 208
42 206
14 270
344 428
61 628
199 568
168 172
209 510
37 540
72 

83 265
149 174
42 510
101 214
139 540
313 352
158 487
218 274
106 253
387 484
109 541
487 612
137 638
157 351
263 327
407 484
205 430
427 540
20 579
107 215
107 134
271 349
109 491
271 351
62 72
371 495
136 273
62 263
107 295
198 218
206 233
459 491
108 371
273 541
278 290
37 571
187 561
41 583
295 352
246 387
309 463
126 449
109 415
427 512
88 162
155 568
342 559
203 371
209 374
389 424
113 146
54 407
231 233
185 532
352 378
33 126
337 469
289 410
19 382
12 20
50 294
83 287
131 614
139 449
113 367
32 431
351 495
337 483
274 339
61 342
274 342
61 339
559 628
287 295
97 461
30 51
190 459
190 458
136 387
490 575
261 457
458 474
272 571
95 391
83 447
326 449
424 622
109 349
45 392
172 305
199 312
97 171
307 373
10 221
392 598
164 501
133 416
334 585
205 539
286 487
137 378
308 351
265 531
426 540
7 459
297 422
164 309
146 371
259 380
335 424
349 521
272 541
187 583
287 457
208 444
216 598
149 184
162 209
162 210
164 550
337 389
51 512
21 460
412 612
100 510
94 293
263 275
17 32
205 401
52

256 419
186 571
90 287
35 220
35 221
220 262
428 586
40 297
322 467
40 298
342 373
107 272
20 550
95 356
190 351
475 496
11 233
344 395
348 467
62 100
262 301
374 430
188 495
392 456
127 425
41 262
113 400
351 424
127 456
127 457
19 475
222 467
155 352
349 467
20 524
407 556
225 533
6 19
190 298
155 217
22 149
191 457
228 312
20 431
240 274
108 144
66 512
412 495
108 322
221 290
327 412
309 334
131 318
290 612
221 305
133 608
161 344
16 336
181 290
259 583
95 269
64 327
361 427
23 467
289 309
228 289
407 495
45 269
351 509
88 492
168 531
134 638
91 460
290 410
91 458
32 363
32 362
187 573
109 420
97 346
282 561
399 492
187 209
162 327
402 635
241 297
53 130
270 636
17 449
108 524
41 572
16 467
42 608
395 415
232 351
310 575
137 346
80 162
142 209
214 627
231 293
182 183
297 407
420 638
100 634
275 415
50 387
328 540
6 351
231 407
233 444
209 220
336 468
61 366
262 395
209 221
270 593
269 273
221 612
132 380
94 351
79 226
79 228
272 487
363 544
52 393
231 345
334 632
108 532
61 582
213 

...

146 524
23 510
383 492
183 186
22 132
41 521
172 274
172 275
162 458
378 490
12 500
415 571
30 337
258 571
37 467
109 607
335 371
125 181
248 473
95 424
76 273
371 415
101 558
45 389
253 262
150 512
125 495
17 95
264 312
161 309
407 612
233 524
205 231
37 505
186 415
222 458
185 460
185 458
36 133
7 475
222 371
100 257
133 342
155 416
270 521
128 295
138 162
20 247
336 422
348 413
172 638
63 163
113 402
15 391
361 460
297 592
235 503
263 337
127 152
290 295
131 217
108 415
400 401
131 468
274 334
369 413
30 461
133 266
37 461
407 510
131 492
127 396
125 126
294 297
14 241
80 467
183 281
43 273
225 231
225 229
335 467
81 352
17 296
17 297
325 397
285 460
209 301
216 271
124 629
297 301
95 106
157 374
287 346
231 314
15 322
274 407
61 496
209 333
322 459
322 458
439 496
27 68
62 461
161 510
40 41
235 435
108 137
113 532
45 416
261 407
149 463
20 490
2 544
41 342
109 314
95 351
41 339
345 388
108 391
295 337
127 190
120 190
310 383
11 370
14 520
37 163
37 162
91 273
41 127
257 408
10 198


72 413
424 540
297 606
43 474
326 352
17 215
23 352
352 490
62 200
397 450
210 461
475 589
17 294
52 181
270 592
126 247
270 595
289 352
37 210
247 373
83 172
401 444
22 458
337 382
27 161
174 412
155 535
127 276
41 301
149 468
334 510
128 487
127 186
80 415
53 352
352 383
172 614
42 124
274 305
169 510
61 122
33 61
37 612
190 231
275 438
290 375
70 294
10 447
161 408
17 68
14 108
186 232
183 415
226 408
52 544
124 374
185 216
185 217
273 568
207 373
262 366
51 378
369 575
157 422
260 319
106 407
251 556
126 271
126 272
42 309
218 233
309 424
308 478
262 487
27 246
8 310
83 199
61 162
61 163
422 467
10 126
27 419
37 361
352 444
95 285
203 351
37 322
157 391
293 352
8 589
5 186
37 526
190 336
108 214
108 215
401 521
45 135
63 606
133 415
150 408
308 455
346 556
45 370
162 275
181 222
36 351
63 622
97 558
413 495
155 387
138 139
352 380
152 383
246 520
17 271
10 401
10 402
7 183
474 556
79 155
327 388
322 510
186 336
220 456
25 222
25 51
101 295
17 625
81 273
243 449
309 473
20 373
146 2

128 568
389 561
22 185
264 290
344 487
185 397
374 575
216 395
10 11
19 510
239 392
305 374
113 352
184 458
109 270
108 434
70 373
94 501
112 374
389 517
182 469
380 448
259 274
490 492
11 64
19 233
19 235
11 67
298 624
106 296
340 369
122 558
61 233
40 294
333 521
262 345
19 467
167 424
274 358
67 214
113 374
272 344
100 126
101 351
374 428
412 614
11 371
209 308
261 309
63 613
8 556
41 240
374 512
133 231
49 444
30 152
36 401
150 444
265 278
351 427
83 638
17 45
131 161
108 467
334 501
133 420
12 217
150 235
186 272
67 94
17 344
17 228
460 560
214 467
327 334
287 414
131 428
49 137
442 638
21 108
36 146
403 413
158 391
80 168
196 274
231 298
131 563
137 188
164 327
309 614
289 510
63 257
150 369
42 403
50 389
281 400
233 449
20 208
54 128
374 595
209 278
382 507
401 486
21 293
94 100
226 428
34 391
34 392
272 488
90 183
90 182
8 334
573 583
27 205
188 369
262 520
229 261
187 358
419 627
158 401
23 146
348 559
271 389
262 519
239 528
157 334
368 378
61 255
313 374
22 126
415 638
274 6

365 559
326 558
274 372
352 401
17 638
352 402
77 346
233 413
72 217
346 374
420 474
91 113
450 613
212 404
235 495
127 226
67 128
237 407
99 336
352 369
7 209
263 388
139 235
32 398
49 401
286 474
351 560
64 531
133 424
205 535
161 228
154 273
429 460
289 295
17 351
298 556
5 346
175 369
174 563
158 287
262 561
157 449
270 555
128 575
390 487
51 441
263 622
128 411
136 281
217 235
217 373
225 395
105 638
203 540
80 136
10 310
368 628
327 556
243 365
390 460
390 459
70 442
380 570
468 592
297 363
138 348
215 297
42 407
373 430
310 458
25 478
336 388
205 478
296 502
275 616
90 199
146 206
149 295
92 287
374 387
191 272
334 448
336 367
231 375
231 374
63 74
88 568
458 475
199 442
149 233
271 392
392 420
376 558
22 251
233 568
126 216
336 438
174 290
17 523
41 216
88 397
97 168
90 413
167 532
309 575
131 199
52 637
166 573
23 183
368 407
221 228
113 318
41 493
126 182
337 387
19 205
41 374
556 627
478 629
332 361
98 166
41 544
228 392
438 460
42 214
51 63
623 638
216 531
131 379
275 374
7

KeyboardInterrupt: 

<h2> Creating adjacency matrices for the drug-protein and protein-drug network.</h2>

In [25]:
drug_gene_adj = np.zeros((n_drugs,n_genes))
gene_drug_adj = np.zeros((n_genes, n_drugs))

In [26]:
for i in range(0,drug_target_protein_data.shape[0]):
    drug = drug_target_protein_data.loc[i][0]#read drug from the drug-gene data
    gene = drug_target_protein_data.loc[i][1]#read gene from the drug-gene data
    gene_index = dict_genes.get(gene)#get the index of gene from the gene dictionary
    drug_index = dict_drugs.get(drug)#ge the indx of drug from the drug dictionary
    drug_gene_adj[drug_index][gene_index] = 1.0
    gene_drug_adj[gene_index][drug_index] = 1.0

In [27]:
adj_mats_orig = {
    (0, 0): [sp.csr_matrix(gene_adj_mat), sp.csr_matrix(gene_adj_mat_t)],
    (0, 1): [sp.csr_matrix(gene_drug_adj)],
    (1, 0): [sp.csr_matrix(drug_gene_adj)],
    (1, 1): drug_drug_adj_list + [x.transpose(copy=True) for x in drug_drug_adj_list],
}

In [28]:
degrees = {
    0: [gene_degrees, gene_degrees],
    1: drug_degrees_list + drug_degrees_list,
}
# featureless (genes)
gene_feat = sp.identity(n_genes)
gene_nonzero_feat, gene_num_feat = gene_feat.shape
gene_feat = preprocessing.sparse_to_tuple(gene_feat.tocoo())
# features (drugs)
drug_feat = sp.identity(n_drugs)
drug_nonzero_feat, drug_num_feat = drug_feat.shape
drug_feat = preprocessing.sparse_to_tuple(drug_feat.tocoo())
# data representation
num_feat = {
    0: gene_num_feat,
    1: drug_num_feat,
}
nonzero_feat = {
    0: gene_nonzero_feat,
    1: drug_nonzero_feat,
}
feat = {
    0: gene_feat,
    1: drug_feat,
}
edge_type2dim = {k: [adj.shape for adj in adjs] for k, adjs in adj_mats_orig.items()}
edge_type2decoder = {
    (0, 0): 'bilinear',
    (0, 1): 'bilinear',
    (1, 0): 'bilinear',
    (1, 1): 'dedicom',
}

In [29]:
edge_types = {k: len(v) for k, v in adj_mats_orig.items()}
num_edge_types = sum(edge_types.values())
print("Edge types:", "%d" % num_edge_types)

###########################################################
#
# Settings and placeholders
#
###########################################################

flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_integer('neg_sample_size', 1, 'Negative sample size.')
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 50, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 64, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 32, 'Number of units in hidden layer 2.')
flags.DEFINE_float('weight_decay', 0, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_float('dropout', 0.1, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('max_margin', 0.1, 'Max margin parameter in hinge loss')
flags.DEFINE_integer('batch_size', 512, 'minibatch size.')
flags.DEFINE_boolean('bias', True, 'Bias term.')
# Important -- Do not evaluate/print validation performance every iteration as it can take
# substantial amount of time
PRINT_PROGRESS_EVERY = 150

print("Defining placeholders")
placeholders = construct_placeholders(edge_types)

Edge types: 34
Defining placeholders


In [30]:
print("Create minibatch iterator")
minibatch = EdgeMinibatchIterator(
    adj_mats=adj_mats_orig,
    feat=feat,
    edge_types=edge_types,
    batch_size=FLAGS.batch_size,
    val_test_size=val_test_size
)

Create minibatch iterator
Minibatch edge type: (0, 0, 0)
Constructing val edges= 0000/0030
Train edges= 0000
Val edges= 0030
Test edges= 0000
Minibatch edge type: (0, 0, 1)
Constructing val edges= 0000/0030
Train edges= 0000
Val edges= 0030
Test edges= 0000
Minibatch edge type: (0, 1, 0)
Constructing val edges= 0000/0006
Train edges= 0000
Val edges= 0006
Test edges= 0000
Minibatch edge type: (1, 0, 0)
Constructing val edges= 0000/0006
Train edges= 0000
Val edges= 0006
Test edges= 0000
Minibatch edge type: (1, 1, 0)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 2)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 3)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 4)
Constructing va

/Users/ravanv/Desktop/Decagon/decagon-master/decagon/deep/minibatch.py:67: RuntimeWarning: divide by zero encountered in power
  rowdegree_mat_inv = sp.diags(np.nan_to_num(np.power(rowsum, -0.5)).flatten())
/Users/ravanv/Desktop/Decagon/decagon-master/decagon/deep/minibatch.py:68: RuntimeWarning: divide by zero encountered in power
  coldegree_mat_inv = sp.diags(np.nan_to_num(np.power(colsum, -0.5)).flatten())


In [31]:
print("Create model")
model = DecagonModel(
    placeholders=placeholders,
    num_feat=num_feat,
    nonzero_feat=nonzero_feat,
    edge_types=edge_types,
    decoders=edge_type2decoder,
)

Create model


In [32]:
print("Create optimizer")
with tf.name_scope('optimizer'):
    opt = DecagonOptimizer(
        embeddings=model.embeddings,
        latent_inters=model.latent_inters,
        latent_varies=model.latent_varies,
        degrees=degrees,
        edge_types=edge_types,
        edge_type2dim=edge_type2dim,
        placeholders=placeholders,
        batch_size=FLAGS.batch_size,
        margin=FLAGS.max_margin
    )

Create optimizer


/Users/ravanv/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [33]:
print("Initialize session")
sess = tf.Session()
sess.run(tf.global_variables_initializer())
feed_dict = {}

Initialize session


In [34]:
###########################################################
#
# Train model
#
###########################################################

print("Train model")
for epoch in range(FLAGS.epochs):

    minibatch.shuffle()
    itr = 0
    while not minibatch.end():
        # Construct feed dictionary
        print("iteration:", itr)
        feed_dict = minibatch.next_minibatch_feed_dict(placeholders=placeholders)
        feed_dict = minibatch.update_feed_dict(
            feed_dict=feed_dict,
            dropout=FLAGS.dropout,
            placeholders=placeholders)

        t = time.time()

        # Training step: run single weight update
        outs = sess.run([opt.opt_op, opt.cost, opt.batch_edge_type_idx], feed_dict=feed_dict)
        train_cost = outs[1]
        batch_edge_type = outs[2]

        if itr % PRINT_PROGRESS_EVERY == 0:
            val_auc, val_auprc, val_apk = get_accuracy_scores(
                minibatch.val_edges, minibatch.val_edges_false,
                minibatch.idx2edge_type[minibatch.current_edge_type_idx])

            print("Epoch:", "%04d" % (epoch + 1), "Iter:", "%04d" % (itr + 1), "Edge:", "%04d" % batch_edge_type,
                  "train_loss=", "{:.5f}".format(train_cost),
                  "val_roc=", "{:.5f}".format(val_auc), "val_auprc=", "{:.5f}".format(val_auprc),
                  "val_apk=", "{:.5f}".format(val_apk), "time=", "{:.5f}".format(time.time() - t))

        itr += 1

print("Optimization finished!")

Train model


KeyboardInterrupt: 

In [ ]:
for et in range(num_edge_types):
    roc_score, auprc_score, apk_score = get_accuracy_scores(
        minibatch.test_edges, minibatch.test_edges_false, minibatch.idx2edge_type[et])
    print("Edge type=", "[%02d, %02d, %02d]" % minibatch.idx2edge_type[et])
    print("Edge type:", "%04d" % et, "Test AUROC score", "{:.5f}".format(roc_score))
    print("Edge type:", "%04d" % et, "Test AUPRC score", "{:.5f}".format(auprc_score))
    print("Edge type:", "%04d" % et, "Test AP@k score", "{:.5f}".format(apk_score))
    print()
